<a href="https://colab.research.google.com/github/emrunali/Clinical_report_based_on_VCF_data_and_ACMG_guidelines/blob/main/ACMG_compliant_clinical_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Step 1: Clone InterVar from its github repository**

In [2]:
!git clone https://github.com/WGLab/InterVar.git

Cloning into 'InterVar'...
remote: Enumerating objects: 697, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 697 (delta 22), reused 34 (delta 16), pack-reused 655
Receiving objects: 100% (697/697), 92.83 MiB | 23.65 MiB/s, done.
Resolving deltas: 100% (390/390), done.


**Step 2: Add mim2gene.txt and morbidmap.txt to /content/intervardb**

In [3]:
%cd InterVar

/content/InterVar


In [4]:
import sys
sys.path.append('/content/InterVar')

**Step 3: Make sure to modify config.ini file in the InterVar folder so that it can access annovar.**

In [9]:
#Before running this cell: download annovar from official website and upload it with humandb folder
!unzip /content/annovar-20240319T032745Z-001.zip -d /content/

Archive:  /content/annovar-20240319T032745Z-001.zip
  inflating: /content/annovar/coding_change.pl  
  inflating: /content/annovar/retrieve_seq_from_fasta.pl  
  inflating: /content/annovar/.DS_Store  
  inflating: /content/annovar/table_annovar.pl  
  inflating: /content/annovar/annotate_variation.pl  
  inflating: /content/annovar/convert2annovar.pl  
  inflating: /content/annovar/example/example.simple_region  
  inflating: /content/annovar/humandb/GRCh37_MT_ensGene.txt  
  inflating: /content/annovar/example/ex1.avinput  
  inflating: /content/annovar/humandb/annovar_downdb.log  
  inflating: /content/annovar/example/example.tab_region  
  inflating: /content/annovar/humandb/hg19_example_db_generic.txt  
  inflating: /content/annovar/humandb/GRCh37_MT_ensGeneMrna.fa  
  inflating: /content/annovar/humandb/hg19_MT_ensGene.txt  
  inflating: /content/annovar/humandb/hg19_refGeneVersion.txt  
  inflating: /content/annovar/example/gene_xref.txt  
  inflating: /content/annovar/variants_

**Step 4: Install PyVCF for parsing VCF files**

In [5]:
!apt-get install python3-vcf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libhts3 libhtscodecs2 python3-pysam
The following NEW packages will be installed:
  libhts3 libhtscodecs2 python3-pysam python3-vcf
0 upgraded, 4 newly installed, 0 to remove and 39 not upgraded.
Need to get 2,235 kB of archives.
After this operation, 7,089 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libhtscodecs2 amd64 1.1.1-3 [53.2 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libhts3 amd64 1.13+ds-2build1 [390 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 python3-pysam amd64 0.17.0+ds-2build1 [1,749 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 python3-vcf amd64 0.6.8+git20170215.476169c-8build1 [43.5 kB]
Fetched 2,235 kB in 0s (15.7 MB/s)
Selecting previously unselected package libhtscodecs2:amd64.
(Reading database ... 12

**Step 5: Unzip VCF file to make it accessible**

In [6]:
!gzip -dk /content/normal_sample.deepvariant.vcf.gz

**Step 6: Filter variant data from the VCF file based on quality, filter status, and variant allele frequency in sample.**

In [7]:
import pysam

def filter_vcf(input_vcf_path, output_vcf_path, min_qual=20, min_vaf=0.3):
    vcf_in = pysam.VariantFile(input_vcf_path)
    vcf_out = pysam.VariantFile(output_vcf_path, 'w', header=vcf_in.header)

    for record in vcf_in.fetch():
        if record.qual < min_qual or record.filter.keys() != ['PASS']:
            continue

        vaf_values = [sample.get('VAF', [0])[0] for sample in record.samples.values()]
        if all(vaf < min_vaf for vaf in vaf_values):
            continue

        vcf_out.write(record)

    vcf_in.close()
    vcf_out.close()

filter_vcf('/content/normal_sample.deepvariant.vcf', '/content/filtered_normal_sample.deepvariant.vcf')


**Step 7: Run Intervar on the filtered VCF file.**

In [10]:
#Run this cell to allow perl executions of Intervar before running it
!chmod +x /content/annovar/*.pl

In [12]:
!python /content/InterVar/Intervar.py \
    --config /content/InterVar/config.ini \
    --input /content/filtered_normal_sample.deepvariant.vcf \
    --output /content/classified_variants \
    --input_type VCF \
    --buildver hg19

InterVar                                                                       
Interpretation of Pathogenic/Benign for variants using python scripts. 

.####.##....##.########.########.########..##.....##....###....########.
..##..###...##....##....##.......##.....##.##.....##...##.##...##.....##
..##..####..##....##....##.......##.....##.##.....##..##...##..##.....##
..##..##.##.##....##....######...########..##.....##.##.....##.########.
..##..##..####....##....##.......##...##....##...##..#########.##...##..
..##..##...###....##....##.......##....##....##.##...##.....##.##....##.
.####.##....##....##....########.##.....##....###....##.....##.##.....##


%prog 2.2.2 20210727
Written by Quan LI,leequan@gmail.com. 
InterVar is free for non-commercial use without warranty.
Please contact the authors for commercial use.
Copyright (C) 2016 Wang Genomic Lab

Notice: Your command of InterVar is ['/content/InterVar/Intervar.py', '--config', '/content/InterVar/config.ini', '--input', '/conte

**Step 8: Install required libraries/packages for report generation**

In [13]:
!pip install reportlab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 14.6 MB/s eta 0:00:00


In [15]:
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.units import inch

In [14]:
import pandas as pd

In [16]:
import re

In [17]:
import vcf

**Step 9: Parse VCF and Intervar files while extracting and storing relevant information in the form of dataframe.**

In [18]:
#Preparing VCF and Intervar dataframes with relevant information from each file for the report

def parse_vcf(vcf_path):
    vcf_data = []
    vcf_in = pysam.VariantFile(vcf_path, 'r')
    for record in vcf_in.fetch():
        qual = round(record.qual,2) if record.qual else 0
        filter_status = ';'.join(list(record.filter.keys())) if record.filter else 'PASS'
        vcf_data.append([record.chrom, str(record.pos), record.ref, ','.join(record.alts), qual, filter_status])
    return pd.DataFrame(vcf_data, columns=['Chromosome', 'Position', 'Reference Allele', 'Alternate Allele', 'Variant Quality', 'Filter Status'])


def extract_acmg_classification(evidence_string):
    known_classifications = ['Benign', 'Pathogenic', 'Likely benign', 'Likely pathogenic', 'Uncertain significance']

    parts = evidence_string.split(';')
    intervar_part = parts[0] if parts else ""
    words = intervar_part.split()
    classification = " ".join(words[1:3]) if len(words) > 1 else "Unknown"

    for known_class in known_classifications:
        if classification.startswith(known_class):
            return known_class

    return classification

def parse_intervar(intervar_path):
    intervar_data = []
    with open(intervar_path, 'r') as f:
        for line in f:
            if line.startswith('#'): continue  # to skip header lines
            parts = line.strip().split('\t')
            chr, start = parts[0], parts[1]
            intervar_evidence = parts[13]
            acmg_classification = extract_acmg_classification(intervar_evidence)
            intervar_data.append([chr, start, acmg_classification])

    return pd.DataFrame(intervar_data, columns=['Chromosome', 'Position', 'ACMG Classification'])

vcf_df = parse_vcf('/content/filtered_normal_sample.deepvariant.vcf')
intervar_df = parse_intervar('/content/classified_variants.hg19_multianno.txt.intervar')
intervar_df['Chromosome'] = intervar_df['Chromosome'].apply(lambda x: 'chr' + x if not x.startswith('chr') else x) #because Intervar file had different type (integers only) of entries in chromosome column


In [19]:
#just to check if everything is in place for report generation
vcf_df_subset = vcf_df.iloc[:50]
intervar_df_subset = intervar_df.iloc[:50]

merged_subset = pd.merge(vcf_df_subset, intervar_df_subset, on=["Chromosome", "Position"], how='inner')

print(merged_subset.head(50))

   Chromosome Position Reference Allele  \
0        chr1    15820                G   
1        chr1    15903                G   
2        chr1    69270                A   
3        chr1    69511                A   
4        chr1    69897                T   
5        chr1   187302                A   
6        chr1   187302                A   
7        chr1   872909                A   
8        chr1   873542                G   
9        chr1   890174                A   
10       chr1   914618                A   
11       chr1   914991                G   
12       chr1   930939                G   
13       chr1   930939                G   
14       chr1   931540                A   
15       chr1   931540                A   
16       chr1   931558                G   
17       chr1   931558                G   
18       chr1   935386                C   
19       chr1   935523                T   
20       chr1   938654                G   
21       chr1   938654                G   
22       ch

**Step 10: Generate PDF report and download it.**

In [20]:
# Extracting the sample ID and gender information from the VCF file
vcf_path = '/content/filtered_normal_sample.deepvariant.vcf'  # make sure to use filtered vcf
vcf_reader = vcf.Reader(open(vcf_path, 'r'))
sample_id = vcf_reader.samples[0] if vcf_reader.samples else "Unknown"

gender = "Female"
for record in vcf_reader:
    if record.CHROM == "Y" or record.CHROM == "chrY":
        gender = "Male"
        break

patient_details = {
    "Patient ID": sample_id,
    "Date of Birth": "1994-03-20",
    "Gender": gender,
    "Ethnicity": "Asian",
    "Family History": "None"
}

#Creating function to wrap text for entries in allele columns that have multiple nucleotides to adjust the table on the page
def wrap_text(text, max_length=9):
    wrapped_text = '\n'.join([text[i:i+max_length] for i in range(0, len(text), max_length)])
    return wrapped_text

# Function to generate PDF including patient details
def generate_pdf(data, filename, patient_info):
    pdf = SimpleDocTemplate(filename, pagesize=letter)
    styles = getSampleStyleSheet()
    elems = []

    for key, value in patient_info.items():
        elems.append(Paragraph(f'<b>{key}:</b> {value}', styles['Normal']))
        elems.append(Spacer(1, 12))

    elems.append(Spacer(1, 20))

    # Preparing data with wrapped text for alleles
    wrapped_data = []
    for row in data:
        wrapped_row = [
            row[0],
            row[1],
            wrap_text(row[2]), #Wrapping reference allele column
            wrap_text(row[3]), #Wrapping alternate allele column
            row[4],
            row[5],
            row[6]
        ]
        wrapped_data.append(wrapped_row)

    variant_table = Table(wrapped_data)
    variant_table.setStyle(TableStyle([
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('GRID', (0,0), (-1,-1), 1, colors.black),
        ('VALIGN', (0,0), (-1,-1), 'TOP'),
    ]))

    elems.append(variant_table)
    pdf.build(elems)

#Checking with subset of data if the PDF being built is in correct format
data_for_pdf = [["Chromosome", "Position", "Reference Allele", "Alternate Allele", "Variant Quality", "Filter Status", "ACMG Classification"]] + merged_subset.values.tolist()

pdf_report_path = '/content/variant_report_with_patient_details.pdf'

generate_pdf(data_for_pdf, pdf_report_path, patient_details)

print("PDF report generated:", pdf_report_path)

#For downloading the PDF file as soon as the file is generated
from google.colab import files
files.download(pdf_report_path)

PDF report generated: /content/variant_report_with_patient_details.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
combined_df = pd.merge(vcf_df, intervar_df, on=["Chromosome", "Position"], how='inner')

full_data_for_pdf = [["Chromosome", "Position", "Reference Allele", "Alternate Allele", "Variant Quality", "Filter Status", "ACMG Classification"]] + combined_df.values.tolist()

pdf_report_path = '/content/full_variant_report_with_patient_details.pdf'

generate_pdf(full_data_for_pdf, pdf_report_path, patient_details)

print("PDF report generated:", pdf_report_path)

from google.colab import files
files.download(pdf_report_path)

PDF report generated: /content/full_variant_report_with_patient_details.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>